# Branch and Bound

In [ ]:
import numpy as np
from itertools import permutations
# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import itertools
from math import *

from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.ising import max_cut, tsp
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit.optimization.ising.common import sample_most_likely
import seaborn as sns

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

In [ ]:
def lowerb(A):
    F = np.zeros((len(A),len(A)))
    v = 0
    s = 0
    for y in range(len(A)):
        for l in range(len(A)):
            F[y,l] = A[y,l]
            
    for i in range (len(A)):
        F[i,i] = 1000000
        w = F[i,:].min()
        v = w + v
        for j in range(len(A)):
            F[i,j] = F[i,j] - w
    F[0,0] = 1000000
    for r in range(len(A)):
        F[r,r] = 1000000
        x = F[:,r].min()
        s = x + s
        for o in range(len(A)):
            F[o,r] = F[o,r] - x
    F[0,0] = 1000000
    return F, s+v

In [ ]:
def reduced(A,a,b):
    B = np.zeros((len(A),len(A)))
    g = 0
    u = 0
    for t in range(1):
        for i in range(len(A)):
            for j in range(len(A)):
                B[i,j] = A[i,j]
                B[i,i] = 1000000
                B[a,j] = 1000000
                B[i,b] = 1000000
                B[a,b] =1000000
                B[b,a] = 1000000
        if B[i,j] <= 50:
            for k in range(len(B)):
                p = B[k,:].min()
                if p < 5000:
                    g = g + p
                for l in range(len(B)):
                    B[k,l] = B[k,l] - p

#                 B[a,k] = 1000
            for m in range(len(B)):
                q = B[:,m].min()
                if q <= 5000:
                    u = u + q
                for n in range(len(B)):
                    if B[n,m] <=5000:
                        B[n,m] = B[n,m] - q 
                        
                        
#             B[m,b] = 1000
#         B[0,b] = 1000

    return B, u+g

In [1]:
def bnb(A):
    overall = []
    for k in range (len(A)):
        new_matrix = lowerb(A)[0]
        lower_bound = lowerb(A)[1]
        M = []
        B = []

        P = []
        ind = 0
        P.append(k)

        for i in range(k,len(A) - 1):
            for j in range(i+1,len(A)):
                new_matrix1 = reduced(new_matrix , i,j)[0]
                bound = reduced(new_matrix,i,j)[1]
                M.append(new_matrix1)
                B.append(bound + A[i,j] + lower_bound)
    #             print(bound + A[i,j] + lower_bound)

            branch = B.index(min(B))
            ind = ind + branch 
            lower_bound = min(B)
            new_matrix = M[branch]

            M = []
            B = []
        lower_bound = lower_bound + A[branch , 0]
        overall.append(lower_bound)
#     print(min(overall))
    return min(overall)